In [1]:
import Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
end
#---

# import necessary packages
using AA228FinalProject
using POMDPs
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf
using QMDP
using JLD

In [2]:
using POMDPModels, BasicPOMCP


┌ Info: Loading Cairo backend into Compose.jl
└ @ Compose /Users/chenyao/.julia/packages/Compose/BYWXX/src/Compose.jl:161
┌ Warning: Package Compose does not have Cairo in its dependencies:
│ - If you have Compose checked out for development and have
│   added Cairo as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Compose
│ Loading Cairo into Compose from project dependency, future warnings for Compose are suppressed.
└ @ nothing nothing:834


In [3]:
sensor = Bumper();
num_x_pts = 50 # e.g. 50
num_y_pts = 50 # e.g. 50
num_th_pts = 20 # e.g. 20
sspace = DiscreteRoombaStateSpace(num_x_pts,num_y_pts,num_th_pts);
vlist = [0, 1, 2, 3]
omlist = [-1, -0.5, 0, 0.5, 1]
aspace = vec(collect(RoombaAct(v, om) for v in vlist, om in omlist));

In [4]:
config = 1 # 1,2, or 3
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config, sspace=sspace, aspace=aspace));

In [5]:
num_particles = 2000
resampler = BumperResampler(num_particles)
spf = SimpleParticleFilter(m, resampler)
v_noise_coefficient = 2.0
om_noise_coefficient = 0.5
belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

In [6]:
solver = POMCPSolver()
POMCP_planner = BasicPOMCP.solve(solver, m);

In [10]:
# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
print("sss")
for (t, step) in enumerate(stepthrough(m, POMCP_planner, belief_updater, max_steps=1000))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        print("sss")
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
end

ssssss

In [8]:
using JLD
JLD.save("QMDP_Bumper_policy.jld", "policy", POMCP_planner)